In [ ]:
import pandas as pd

# 读取数据
csv_path = r"E:\git\DataScience_Project_Team_25\data\pre_transform\Age group_gender.csv"
df = pd.read_csv(csv_path)

# 以 'feature' 为例，统计概率分布和加权平均收入
group_col = 'feature'
weight_col = 'number_of_jobs'
income_col = 'median_income'

# 概率分布
prob = df.groupby(group_col)[weight_col].sum()
prob = prob / prob.sum()

# 加权平均收入
def weighted_mean(x):
    return (x[weight_col] * x[income_col]).sum() / x[weight_col].sum()

mean_income = df.groupby(group_col).apply(weighted_mean)

# 合并结果
stats = pd.DataFrame({
    'prob': prob,
    'mean_income': mean_income
})

print(stats)

In [ ]:
import pandas as pd

def extract_feature_tables_auto(excel_path, sheet_name=0, skiprows=6, feature_col='Feature', sex_col='Sex'):
    df = pd.read_excel(excel_path, sheet_name=sheet_name, skiprows=skiprows)
    cols = df.columns.tolist()
    class_idx = cols.index(feature_col)
    sex_idx = cols.index(sex_col)
    value_cols = cols[max(class_idx, sex_idx)+1:]
    selected_cols = [feature_col, sex_col, "State Name"] + value_cols
    result_df = df[selected_cols].copy()
    result_df = result_df.dropna(how='all').reset_index(drop=True)
    for col in value_cols:
        result_df = result_df[pd.to_numeric(result_df[col], errors='coerce').notnull()]
    n_values = len(value_cols)
    half = n_values // 2
    jobs_cols = value_cols[:half]
    income_cols = value_cols[half:]

    records = []
    for _, row in result_df.iterrows():
        if row["State Name"] == "Australia":
            feature = row[feature_col]
            sex = row[sex_col]
            for i, year in enumerate(jobs_cols):
                records.append({
                    'feature': feature,
                    'year': year,
                    'gender': sex,
                    'number_of_jobs': float(row[year]) / 1000,
                    'median_income': row[income_cols[i]]
                })
    return pd.DataFrame(records)